## OSEMN Approach
1 - Obtaining the data 2 - Scrubbing the data 3 - Exploring the data 4 - Modeling the data 5 - iNterpreting the results


![title](picture2.png)


## Questions:

1) What are the main factors to predict the price of a house?

2) How much does a middle income family should invest to buy a standard house?
   
   or What are the properties of a house you can buy in King County if your budget is $400,000???
    
3) How can a middle income family increase the price of their house by %20 by spending %5 of the house cost???

4) Basement
 

In [1]:
%autosave 10

Autosaving every 10 seconds


In [2]:
# # Print all the interactive output without resorting to print
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

## Import the necessary packages

First, we’ll import all of our necessary packages.

In [3]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
# import sklearn as sk 
from statsmodels.formula.api import ols


In [4]:
# ## Display more information 
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)

## Import the data set

Next, we’ll import our data and see what we’re working with. 

In [5]:
df=pd.read_csv('kc_house_data.csv')

Lets see what the data looks like by functions head(), tail(), shape(), info() and describe().

In [6]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


### Drop unnecaessary columns

In [7]:
df=df.drop(['id','date'],axis=1)

# Scraping and cleaning data

In [8]:
df.shape

(21597, 19)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 19 columns):
price            21597 non-null float64
bedrooms         21597 non-null int64
bathrooms        21597 non-null float64
sqft_living      21597 non-null int64
sqft_lot         21597 non-null int64
floors           21597 non-null float64
waterfront       19221 non-null float64
view             21534 non-null float64
condition        21597 non-null int64
grade            21597 non-null int64
sqft_above       21597 non-null int64
sqft_basement    21597 non-null object
yr_built         21597 non-null int64
yr_renovated     17755 non-null float64
zipcode          21597 non-null int64
lat              21597 non-null float64
long             21597 non-null float64
sqft_living15    21597 non-null int64
sqft_lot15       21597 non-null int64
dtypes: float64(8), int64(10), object(1)
memory usage: 3.1+ MB


### Numerical Data Stored as Strings

In [10]:
df.sqft_basement.value_counts()

0.0       12826
?           454
600.0       217
500.0       209
700.0       208
          ...  
2360.0        1
2350.0        1
588.0         1
1245.0        1
666.0         1
Name: sqft_basement, Length: 304, dtype: int64

In [11]:
df['sqft_basement']=df['sqft_basement'].map(lambda x:float(x.replace('?', '0')))


In [12]:
#Replace 0 with difference between sqft_above and sqft_living
df['sqft_basement'].replace(to_replace='0', value = abs(df['sqft_living']-df['sqft_above']), inplace=True)


In [13]:
df['bedrooms'].replace(to_replace='33', value = 3, inplace=True)


In [14]:
df.describe().round(1)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,21597.0,21597.0,21597.0,21597.0,21597.0,21597.0,19221.0,21534.0,21597.0,21597.0,21597.0,21597.0,21597.0,17755.0,21597.0,21597.0,21597.0,21597.0,21597.0
mean,540296.6,3.4,2.1,2080.3,15099.4,1.5,0.0,0.2,3.4,7.7,1788.6,285.7,1971.0,83.6,98078.0,47.6,-122.2,1986.6,12758.3
std,367368.1,0.9,0.8,918.1,41412.6,0.5,0.1,0.8,0.7,1.2,827.8,439.8,29.4,399.9,53.5,0.1,0.1,685.2,27274.4
min,78000.0,1.0,0.5,370.0,520.0,1.0,0.0,0.0,1.0,3.0,370.0,0.0,1900.0,0.0,98001.0,47.2,-122.5,399.0,651.0
25%,322000.0,3.0,1.8,1430.0,5040.0,1.0,0.0,0.0,3.0,7.0,1190.0,0.0,1951.0,0.0,98033.0,47.5,-122.3,1490.0,5100.0
50%,450000.0,3.0,2.2,1910.0,7618.0,1.5,0.0,0.0,3.0,7.0,1560.0,0.0,1975.0,0.0,98065.0,47.6,-122.2,1840.0,7620.0
75%,645000.0,4.0,2.5,2550.0,10685.0,2.0,0.0,0.0,4.0,8.0,2210.0,550.0,1997.0,0.0,98118.0,47.7,-122.1,2360.0,10083.0
max,7700000.0,33.0,8.0,13540.0,1651359.0,3.5,1.0,4.0,5.0,13.0,9410.0,4820.0,2015.0,2015.0,98199.0,47.8,-121.3,6210.0,871200.0


### Buraya describe ile ilgili yorum yaz
Most houses were build around 1975; sold for $450K; had square footage of living spaces around 1910. You can also note, that distributions of price and variables such as sqft_lot are skewed to the upper tail.



## Detecting and Dealing With Null Values

In [15]:
print(df.isna().sum())

price               0
bedrooms            0
bathrooms           0
sqft_living         0
sqft_lot            0
floors              0
waterfront       2376
view               63
condition           0
grade               0
sqft_above          0
sqft_basement       0
yr_built            0
yr_renovated     3842
zipcode             0
lat                 0
long                0
sqft_living15       0
sqft_lot15          0
dtype: int64


### Buraya na values  ile ilgili yorum yaz


# waterfront ve view 

In [16]:
df['view'].value_counts()

0.0    19422
2.0      957
3.0      508
1.0      330
4.0      317
Name: view, dtype: int64

In [17]:
df['view'].isna().sum()

63

In [18]:
feats = ['view', 'waterfront']
df[feats].corr()

,view,waterfront
view,1.000000,0.406654
waterfront,0.406654,1.000000


In [19]:
print('Number missing both:',
      len(df[(df.view.isnull())
      & (df.waterfront.isnull())]))

Number missing both: 6


In [20]:
len(df[(df.view==2)&(df.waterfront.isna())])

121

In [21]:
len(df[(df.view==3)&(df.waterfront>0)])

14

In [22]:
df['view'].fillna(0,inplace=True)

In [23]:
df['waterfront'].value_counts()

0.0    19075
1.0      146
Name: waterfront, dtype: int64

In [24]:
df['waterfront'].isna().sum()

2376

In [25]:
df['waterfront'].fillna(0,inplace=True)

In [26]:
feats = ['view', 'waterfront']
df[feats].corr()

,view,waterfront
view,1.000000,0.380543
waterfront,0.380543,1.000000


In [27]:
df.yr_renovated.fillna(0, inplace=True)

### Outliers

In [28]:
Q1 = df.quantile(0.005)
Q3 = df.quantile(0.98)
IQR = Q3 - Q1

df_no_outliers = df[~((df < (Q1 - 1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

## Normalizing the data

In [29]:
def norm_feat(series):
    return (series - series.mean())/series.std()
for feat in ['bathrooms', 'sqft_living', 
       'sqft_lot','sqft_above', 'sqft_basement', 'yr_built','yr_renovated','sqft_living15', 'sqft_lot15']:
    df[feat] = norm_feat(df[feat])


In [30]:
df.describe().round(2)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00,21597.00
mean,540296.57,3.37,-0.00,-0.00,0.00,1.49,0.01,0.23,3.41,7.66,0.00,-0.00,0.00,-0.00,98077.95,47.56,-122.21,-0.00,-0.00
std,367368.14,0.93,1.00,1.00,1.00,0.54,0.08,0.76,0.65,1.17,1.00,1.00,1.00,1.00,53.51,0.14,0.14,1.00,1.00
min,78000.00,1.00,-2.10,-1.86,-0.35,1.00,0.00,0.00,1.00,3.00,-1.71,-0.65,-2.42,-0.19,98001.00,47.16,-122.52,-2.32,-0.44
25%,322000.00,3.00,-0.48,-0.71,-0.24,1.00,0.00,0.00,3.00,7.00,-0.72,-0.65,-0.68,-0.19,98033.00,47.47,-122.33,-0.72,-0.28
50%,450000.00,3.00,0.17,-0.19,-0.18,1.50,0.00,0.00,3.00,7.00,-0.28,-0.65,0.14,-0.19,98065.00,47.57,-122.23,-0.21,-0.19
75%,645000.00,4.00,0.50,0.51,-0.11,2.00,0.00,0.00,4.00,8.00,0.51,0.60,0.89,-0.19,98118.00,47.68,-122.12,0.54,-0.10
max,7700000.00,33.00,7.65,12.48,39.51,3.50,1.00,4.00,5.00,13.00,9.21,10.31,1.50,5.35,98199.00,47.78,-121.32,6.16,31.47


In [31]:
df.to_csv("king_county_dataset_cleaned.csv", index=False)


## One-Hot Encoding Categorical Columns


#### Identifying Categorical Variables

These columns can be thought as categorical values. Bedrooms, floors, waterfront, wiev, grade, zipcode and year renovated.
Let's explore them one by one.

In [32]:
df['bedrooms'] = df['bedrooms'].astype('category')
df['floors'] = df['floors'].astype('category')
df['waterfront'] = df['waterfront'].astype('category')
df['view'] = df['view'].astype('category')
df['condition'] = df['condition'].astype('category')
df['grade'] = df['grade'].astype('category')


In [33]:
feats = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 
       'sqft_living15', 'sqft_lot15']
df_feats = df[feats]
df_feats = pd.get_dummies(df_feats, drop_first=True)

In [ ]:
df_feats.info()

In [ ]:
df_feats.head()

'zipcode', 'lat', 'long',

# Exploring the data

In [ ]:
df.head()

In [ ]:
df_feats.head()

In [ ]:
sns.pairplot(df)

## Check for Linearity

jointplots

## Check for Multicollinearity

In [ ]:
# df_target = df_no_outliers.loc['price']
df_heatmap=df.drop(['price','zipcode', 'lat', 'long'],axis=1)
plt.figure(figsize=(20,20))
ax=sns.heatmap(df_heatmap.corr(), center=0, linewidths=.5,annot=True);
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)
plt.show()

In [ ]:
df_heatmap.corr() > 0.75


In [ ]:
df_feats.info()

In [ ]:
# # df=df.drop(['sqft_living'], axis=1)
# df_feats=df_feats.drop(['sqft_living'], axis=1)
# df_feats=df_feats.drop(['sqft_living'], axis=1)

In [ ]:
df_feats.columns

In [43]:
outcome = 'price'
x_cols = ['bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15',
       'sqft_lot15', 'bedrooms_2', 'bedrooms_3', 'bedrooms_4', 'bedrooms_5',
       'bedrooms_6', 'bedrooms_7', 'bedrooms_8', 'bedrooms_9', 'bedrooms_10',
       'bedrooms_11', 'bedrooms_33', 'floors_1_5', 'floors_2_0', 'floors_2_5',
       'floors_3_0', 'floors_3_5', 'waterfront_1_0', 'view_1_0', 'view_2_0',
       'view_3_0', 'view_4_0', 'condition_2', 'condition_3', 'condition_4',
       'condition_5', 'grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8',
       'grade_9', 'grade_10', 'grade_11', 'grade_12', 'grade_13']

In [37]:
# df_feats.columns = [col.replace('.', '_', inplace=True) for col in df_feats.columns]
# df_feats.columns = [col.replace(' ', '_', inplace=True) for col in df_feats.columns]
df_feats.columns = df_feats.columns.str.replace(r"[.]", "_")

In [40]:
df_feats.columns

Index(['price', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15',
       'sqft_lot15', 'bedrooms_2', 'bedrooms_3', 'bedrooms_4', 'bedrooms_5',
       'bedrooms_6', 'bedrooms_7', 'bedrooms_8', 'bedrooms_9', 'bedrooms_10',
       'bedrooms_11', 'bedrooms_33', 'floors_1_5', 'floors_2_0', 'floors_2_5',
       'floors_3_0', 'floors_3_5', 'waterfront_1_0', 'view_1_0', 'view_2_0',
       'view_3_0', 'view_4_0', 'condition_2', 'condition_3', 'condition_4',
       'condition_5', 'grade_4', 'grade_5', 'grade_6', 'grade_7', 'grade_8',
       'grade_9', 'grade_10', 'grade_11', 'grade_12', 'grade_13'],
      dtype='object')

In [44]:
# Fitting the actual model
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=df_feats).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.688
Model:                            OLS   Adj. R-squared:                  0.687
Method:                 Least Squares   F-statistic:                     1078.
Date:                Tue, 21 Apr 2020   Prob (F-statistic):               0.00
Time:                        18:48:21   Log-Likelihood:            -2.9483e+05
No. Observations:               21597   AIC:                         5.897e+05
Df Residuals:                   21552   BIC:                         5.901e+05
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       2.882e+05   2.09e+05      1.377      0.168   -1.22e+05    6.98e+05
bathrooms       3.913e+04   2599.981     15.050      0.000     3.4e+04    4.42e+04
sqft_living     9.211e+04    1.7e+04      5.427      0.000    5.88e+04    1.25e+05
sqft_lot        -147.6370   2024.181     -0.073      0.942   -4115.182    3819.908
sqft_above      1.979e+04   1.53e+04      1.292      0.197   -1.02e+04    4.98e+04
sqft_basement   1.858e+04   8051.201      2.308      0.021    2801.487    3.44e+04
yr_built       -9.023e+04   2220.324    -40.638      0.000   -9.46e+04   -8.59e+04
yr_renovated    9082.1491   1488.038      6.103      0.000    6165.484     1.2e+04
sqft_living15   2.584e+04   2399.099     10.772      0.000    2.11e+04    3.05e+04
sqft_lot15     -1.517e+04   2037.956     -7.444      0.000   -1.92e+04   -1.12e+04
bedrooms_2      9328.4538   1.56e+04      0.598      0.550   -2.12e+04    3.99e+04
bedrooms_3     -4.266e+04   1.55e+04     -2.745      0.006   -7.31e+04   -1.22e+04
bedrooms_4     -7.958e+04   1.59e+04     -5.013      0.000   -1.11e+05   -4.85e+04
bedrooms_5     -6.882e+04   1.68e+04     -4.102      0.000   -1.02e+05   -3.59e+04
bedrooms_6     -1.024e+05   2.05e+04     -4.995      0.000   -1.43e+05   -6.22e+04
bedrooms_7      -1.41e+05   3.75e+04     -3.760      0.000   -2.14e+05   -6.75e+04
bedrooms_8      6.733e+04   5.98e+04      1.125      0.261      -5e+04    1.85e+05
bedrooms_9     -2.343e+05    8.6e+04     -2.723      0.006   -4.03e+05   -6.56e+04
bedrooms_10    -9.754e+04    1.2e+05     -0.813      0.416   -3.33e+05    1.38e+05
bedrooms_11    -3.317e+05   2.06e+05     -1.607      0.108   -7.36e+05    7.28e+04
bedrooms_33     1.218e+05   2.06e+05      0.591      0.555   -2.82e+05    5.26e+05
floors_1_5      1.827e+04   5575.178      3.277      0.001    7340.353    2.92e+04
floors_2_0      1.548e+04   4514.711      3.429      0.001    6630.168    2.43e+04
floors_2_5      1.346e+05   1.69e+04      7.981      0.000    1.02e+05    1.68e+05
floors_3_0      1.385e+05   9581.692     14.451      0.000     1.2e+05    1.57e+05
floors_3_5      1.818e+05   7.83e+04      2.321      0.020    2.82e+04    3.35e+05
waterfront_1_0  5.319e+05   2.09e+04     25.497      0.000    4.91e+05    5.73e+05
view_1_0        1.092e+05   1.16e+04      9.447      0.000    8.65e+04    1.32e+05
view_2_0        4.609e+04   7019.182      6.566      0.000    3.23e+04    5.98e+04
view_3_0        9.752e+04   9604.744     10.153      0.000    7.87e+04    1.16e+05
view_4_0        2.327e+05   1.46e+04     15.992      0.000    2.04e+05    2.61e+05
condition_2     2.624e+04   4.14e+04      0.633      0.527    -5.5e+04    1.07e+05
condition_3     5.174e+04   3.86e+04      1.342      0.180   -2.38e+04    1.27e+05
condition_4     6.995e+04   3.86e+04      1.813      0.070   -5655.767    1.46e+05
condition_5 

## notes


basement to new feature



subsampling

regularisation 



future work:  garaj, pool, okul gradeleri

## web sites
hexbins with longitude and latitude 
https://medium.com/@mattheweparker/visualizing-data-with-hexbins-in-python-39823f89525e  

categorigal data 
https://medium.com/@rabinpoudyal1995/encoding-categorical-data-in-python-fab150d6e21b

All houses are sold within 390 days, therefore I will not need to adjust the price.
https://github.com/Burton-David/KingsCountyHousingLR/blob/master/student.ipynb




https://www.incomebyzipcode.com/washington/98109

https://medium.com/@allisonkelly42/exploratory-data-analysis-with-the-king-county-housing-data-c6d3f7eadc57